In [9]:
import pickle
import numpy as np
import matplotlib.pyplot as plot
import os
import pandas as pd


In [3]:
# for push to git, add files and remove paths..

pickledir='/home/kallanved/tracking/deeplabcut/'
os.chdir(pickledir)
fname="trial_ 15DeepCut_resnet50_With_OcclusionsJun7shuffle1_600000includingmetadata.pickle"
import pickle
with open(fname, "rb") as input_file:
    e = pickle.load(input_file)
    
    


In [5]:
## Write this as a function..


scmaplocref=e['scmaplocref']
scmaplocref.keys()
#scmaplocref=scmaplocref[3]
scmap=scmaplocref['scmap']
locref=scmaplocref['locref']


(64, 192, 12, 2)

In [6]:
ny,nx,num_joints = scmap.shape
    
    #Combine scoremat and offsets to the final pose.
LOCREF=locref.reshape(nx*ny,num_joints,2)
LOCREF.shape

scmap.shape


(64, 192, 12)

In [10]:
screshaped=scmap.reshape((nx*ny,num_joints))
MAXLOC = np.argmax(screshaped,axis=1)
print(scmap.shape,screshaped.shape,MAXLOC.shape)
print(MAXLOC)

(64, 192, 12) (12288, 12) (12288,)
[ 1  3  7 ... 10 10 10]


In [12]:
fullmap = np.empty((nx*ny,3*num_joints)
for ii in range(0,nx*ny): 
    MAXLOC_=ii*np.ones(num_joints)
    Y,X=np.unravel_index(MAXLOC_,dims=(ny,nx))
    DZ=np.zeros((batchsize,num_joints,3))
    for l in range(batchsize):
        for k in range(num_joints):
            DZ[l,k,:2]=LOCREF[l,MAXLOC_[l,k],k,:]
            DZ[l,k,2]=scmap[l,Y[l,k],X[l,k],k]

    X=X.astype('float32')*cfg.stride+.5*cfg.stride+DZ[:,:,0]
    Y=Y.astype('float32')*cfg.stride+.5*cfg.stride+DZ[:,:,1]
    fullmap[ii,0::3] = X
    fullmap[ii,1::3] = Y
    fullmap[ii,2::3] = DZ[:,:,2]
    


SyntaxError: invalid syntax (<ipython-input-12-8e04fab96a1d>, line 2)

In [ ]:
nanX, nanY = arg(isnan(fullmap))
predictions = 2Dinterpolate(fullmap[:,2::3],fullmap[:,0::3],fullmap[:,1::3],nanX,nanY)
fullmap(nanX,nanY) = predictions
# this code is for interpolation between kernel areas 

In [ ]:
### Visualizations
